In [2]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_id.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,305502,3132717110,4/11/20,1087,PPP,THOMAS MANAGEMENT LLC,700 E FRANKLIN RD,MERIDIAN,ID,83642.0,...,999 Other,7223 Special Food Services,7223.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"700 E FRANKLIN RD, MERIDIAN, ID",43.605187,-116.382836,NaN
1,305503,4037657101,4/12/20,1087,PPP,"CRADLEPOINT, INC.",1111 JEFFERSON ST,BOISE,ID,83702.0,...,334 Computers and Electronic Products,3342 Communications Equipment Manufacturing,3342.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"1111 JEFFERSON ST, BOISE, ID",43.619731,-116.205940,NaN
2,305504,4034667109,4/12/20,1087,PPP,"ENGINEERED STRUCTURES, INC.","3330 E. Louise Dr, Suite 300",MERIDIAN,ID,83642.0,...,999 Other,2362 Nonresidential Building Construction,2362.0,BOISE DISTRICT OFFICE,e $5-10 million,250 to 499,"3330 E. Louise Dr, Suite 300, MERIDIAN, ID",43.604024,-116.353172,NaN
3,305505,3029957103,4/11/20,1087,PPP,"COMMERCIAL TIRE, INC.",2095 E. Commercial Street,MERIDIAN,ID,83642.0,...,999 Other,"4413 Automotive Parts, Accessories, and Tire S...",4413.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"2095 E. Commercial Street, MERIDIAN, ID",43.609116,-116.368114,NaN
4,305506,4086377107,4/12/20,1087,PPP,"J-U-B ENGINEERS, INC.","250 S. Beechwood Ave., #201",BOISE,ID,83709.0,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,BOISE DISTRICT OFFICE,e $5-10 million,250 to 499,"250 S. Beechwood Ave., #201, BOISE, ID",43.602477,-116.272347,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4411,309913,8528738503,3/10/21,1094,PPS,WHITCO INC.,108 Airport Rd,Kamiah,ID,83536.0,...,999 Other,4842 Specialized Freight Trucking,4842.0,SPOKANE BRANCH OFFICE,"a $150,000-350,000",5 to 9,"108 Airport Rd, Kamiah, ID",46.218843,-116.016474,NaN
4412,309914,8585757008,4/8/20,1094,PPP,"POST FALLS FAMILY DENTAL CENTER, PA",313 N Spokane Street,POST FALLS,ID,83854.0,...,999 Other,6212 Offices of Dentists,6212.0,SPOKANE BRANCH OFFICE,"a $150,000-350,000",10 to 19,"313 N Spokane Street, POST FALLS, ID",47.710015,-116.948526,NaN
4413,309915,9002167110,4/15/20,1087,PPP,ROBERTS ELECTRIC INC,PO BOX 3869,HAILEY,ID,83333.0,...,999 Other,2382 Building Equipment Contractors,2382.0,BOISE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"PO BOX 3869, HAILEY, ID",43.519629,-114.315325,NaN
4414,309916,9060577102,4/15/20,1094,PPP,DEI INCORPORATED,13403 W GOVERNMENT WAY,HAYDEN,ID,83835.0,...,999 Other,2382 Building Equipment Contractors,2382.0,SPOKANE BRANCH OFFICE,"a $150,000-350,000",20 to 49,"13403 W GOVERNMENT WAY, HAYDEN, ID",47.793812,-116.787280,NaN


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long
2479,1944747104,"PO Box 39, STAR, ID",NaN,NaN
2968,3378157108,"208 Spruce Ave Unit B - 4c, KETCHUM, ID",NaN,NaN


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,3132717110,"700 E FRANKLIN RD, MERIDIAN, ID",43.605187,-116.382836
1,4037657101,"1111 JEFFERSON ST, BOISE, ID",43.619731,-116.205940
2,4034667109,"3330 E. Louise Dr, Suite 300, MERIDIAN, ID",43.604024,-116.353172
3,3029957103,"2095 E. Commercial Street, MERIDIAN, ID",43.609116,-116.368114
4,4086377107,"250 S. Beechwood Ave., #201, BOISE, ID",43.602477,-116.272347


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [3]:
fips_check_scraped = pd.read_csv("../../data/state_data/geo/geo_fips/ID_fips_scraped.csv") 
fips_check_scraped.head() #15869

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,3132717110,4/11/20,1087,PPP,THOMAS MANAGEMENT LLC,700 E FRANKLIN RD,MERIDIAN,ID,83642.0,...,999 Other,7223 Special Food Services,7223.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"700 E FRANKLIN RD, MERIDIAN, ID",43.605187,-116.382836,1.600101e+14
1,1,4037657101,4/12/20,1087,PPP,"CRADLEPOINT, INC.",1111 JEFFERSON ST,BOISE,ID,83702.0,...,334 Computers and Electronic Products,3342 Communications Equipment Manufacturing,3342.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"1111 JEFFERSON ST, BOISE, ID",43.619731,-116.205940,1.600100e+14
2,2,4034667109,4/12/20,1087,PPP,"ENGINEERED STRUCTURES, INC.","3330 E. Louise Dr, Suite 300",MERIDIAN,ID,83642.0,...,999 Other,2362 Nonresidential Building Construction,2362.0,BOISE DISTRICT OFFICE,e $5-10 million,250 to 499,"3330 E. Louise Dr, Suite 300, MERIDIAN, ID",43.604024,-116.353172,1.600101e+14
3,3,3029957103,4/11/20,1087,PPP,"COMMERCIAL TIRE, INC.",2095 E. Commercial Street,MERIDIAN,ID,83642.0,...,999 Other,"4413 Automotive Parts, Accessories, and Tire S...",4413.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"2095 E. Commercial Street, MERIDIAN, ID",43.609116,-116.368114,1.600101e+14
4,4,4086377107,4/12/20,1087,PPP,"J-U-B ENGINEERS, INC.","250 S. Beechwood Ave., #201",BOISE,ID,83709.0,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,BOISE DISTRICT OFFICE,e $5-10 million,250 to 499,"250 S. Beechwood Ave., #201, BOISE, ID",43.602477,-116.272347,1.600100e+14


In [4]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [5]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [6]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

,LoanNumber,full_add,Lat,Long,FIPS_z
0,3132717110,"700 E FRANKLIN RD, MERIDIAN, ID",43.605187,-116.382836,1.600101e+14
1,4037657101,"1111 JEFFERSON ST, BOISE, ID",43.619731,-116.205940,1.600100e+14
2,4034667109,"3330 E. Louise Dr, Suite 300, MERIDIAN, ID",43.604024,-116.353172,1.600101e+14
3,3029957103,"2095 E. Commercial Street, MERIDIAN, ID",43.609116,-116.368114,1.600101e+14
4,4086377107,"250 S. Beechwood Ave., #201, BOISE, ID",43.602477,-116.272347,1.600100e+14
...,...,...,...,...,...
4411,8528738503,"108 Airport Rd, Kamiah, ID",46.218843,-116.016474,NaN
4412,8585757008,"313 N Spokane Street, POST FALLS, ID",47.710015,-116.948526,NaN
4413,9002167110,"PO BOX 3869, HAILEY, ID",43.519629,-114.315325,NaN
4414,9060577102,"13403 W GOVERNMENT WAY, HAYDEN, ID",47.793812,-116.787280,NaN


In [7]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #5503

,LoanNumber,full_add,Lat,Long,FIPS_z
369,5223057107,"1805 SW 4th Ave, FRUITLAND, ID",44.025265,-116.991226,NaN
904,8767187200,"7556 W. Diagonal Road, RATHDRUM, ID",47.828363,-116.867626,NaN
905,1228297208,"PO Box 125, EMMETT, ID",43.873498,-116.499301,NaN
906,6342957005,"1842 E. Plaza Loop, NAMPA, ID",43.591065,-116.549476,NaN
907,3555387207,"601 E Karcher Rd, NAMPA, ID",43.604403,-116.565470,NaN
...,...,...,...,...,...
4411,8528738503,"108 Airport Rd, Kamiah, ID",46.218843,-116.016474,NaN
4412,8585757008,"313 N Spokane Street, POST FALLS, ID",47.710015,-116.948526,NaN
4413,9002167110,"PO BOX 3869, HAILEY, ID",43.519629,-114.315325,NaN
4414,9060577102,"13403 W GOVERNMENT WAY, HAYDEN, ID",47.793812,-116.787280,NaN


<h4>FIPS Script

In [8]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [9]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
#driver = webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))
#executable_path = {'executable_path': webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))}
executable_path = {'executable_path': ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()}
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', **executable_path, options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('16')] #ID
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/3511 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [10]:
geo_id_list =geo_id 

Length Check

In [34]:
print(len(geo_id_list))

611


Matching FIPS to DataFrame

In [11]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:293] #confirm pair to notna DataFrame
#if FIPS available
fips_fill = fips_check_scraped_null

In [12]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LoanNumber,full_add,Lat,Long,FIPS_z
369,5223057107,"1805 SW 4th Ave, FRUITLAND, ID",44.025265,-116.991226,[]
904,8767187200,"7556 W. Diagonal Road, RATHDRUM, ID",47.828363,-116.867626,[160550003042001]
905,1228297208,"PO Box 125, EMMETT, ID",43.873498,-116.499301,[160459603022031]
906,6342957005,"1842 E. Plaza Loop, NAMPA, ID",43.591065,-116.549476,[160270204043030]
907,3555387207,"601 E Karcher Rd, NAMPA, ID",43.604403,-116.565470,[160270204043012]
...,...,...,...,...,...
4411,8528738503,"108 Airport Rd, Kamiah, ID",46.218843,-116.016474,[160499400002003]
4412,8585757008,"313 N Spokane Street, POST FALLS, ID",47.710015,-116.948526,[160550004024040]
4413,9002167110,"PO BOX 3869, HAILEY, ID",43.519629,-114.315325,[160139601023008]
4414,9060577102,"13403 W GOVERNMENT WAY, HAYDEN, ID",47.793812,-116.787280,[160550002022016]


Remove brackets

In [13]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LoanNumber,full_add,Lat,Long,FIPS_z
369,5223057107,"1805 SW 4th Ave, FRUITLAND, ID",44.025265,-116.991226,NaN
904,8767187200,"7556 W. Diagonal Road, RATHDRUM, ID",47.828363,-116.867626,160550003042001
905,1228297208,"PO Box 125, EMMETT, ID",43.873498,-116.499301,160459603022031
906,6342957005,"1842 E. Plaza Loop, NAMPA, ID",43.591065,-116.549476,160270204043030
907,3555387207,"601 E Karcher Rd, NAMPA, ID",43.604403,-116.565470,160270204043012
...,...,...,...,...,...
4411,8528738503,"108 Airport Rd, Kamiah, ID",46.218843,-116.016474,160499400002003
4412,8585757008,"313 N Spokane Street, POST FALLS, ID",47.710015,-116.948526,160550004024040
4413,9002167110,"PO BOX 3869, HAILEY, ID",43.519629,-114.315325,160139601023008
4414,9060577102,"13403 W GOVERNMENT WAY, HAYDEN, ID",47.793812,-116.787280,160550002022016


In [14]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
369,5223057107,"1805 SW 4th Ave, FRUITLAND, ID",44.025265,-116.991226,NaN
1879,6559698606,"20158 Highway 93, Carey, ID",46.678546,-114.074737,NaN
2296,6866047004,"20158 hwy 93, CAREY, ID",46.678546,-114.074737,NaN


Parse for Merge

In [15]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
369,5223057107,NaN
904,8767187200,160550003042001
905,1228297208,160459603022031
906,6342957005,160270204043030
907,3555387207,160270204043012
...,...,...
4411,8528738503,160499400002003
4412,8585757008,160550004024040
4413,9002167110,160139601023008
4414,9060577102,160550002022016


Read in base data

In [17]:
#if from geocode file
#fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_id.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
#fips_check.head()

#if from FIPS file
fips_check = pd.read_csv("../../data/state_data/geo/geo_fips/ID_fips_scraped.csv") 
fips_check = fips_check.drop([ "Unnamed: 0"], axis =1) 
fips_check.head() #22826


,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,3132717110,4/11/20,1087,PPP,THOMAS MANAGEMENT LLC,700 E FRANKLIN RD,MERIDIAN,ID,83642.0,11/10/21,...,999 Other,7223 Special Food Services,7223.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"700 E FRANKLIN RD, MERIDIAN, ID",43.605187,-116.382836,1.600101e+14
1,4037657101,4/12/20,1087,PPP,"CRADLEPOINT, INC.",1111 JEFFERSON ST,BOISE,ID,83702.0,10/21/20,...,334 Computers and Electronic Products,3342 Communications Equipment Manufacturing,3342.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"1111 JEFFERSON ST, BOISE, ID",43.619731,-116.205940,1.600100e+14
2,4034667109,4/12/20,1087,PPP,"ENGINEERED STRUCTURES, INC.","3330 E. Louise Dr, Suite 300",MERIDIAN,ID,83642.0,7/15/21,...,999 Other,2362 Nonresidential Building Construction,2362.0,BOISE DISTRICT OFFICE,e $5-10 million,250 to 499,"3330 E. Louise Dr, Suite 300, MERIDIAN, ID",43.604024,-116.353172,1.600101e+14
3,3029957103,4/11/20,1087,PPP,"COMMERCIAL TIRE, INC.",2095 E. Commercial Street,MERIDIAN,ID,83642.0,6/17/21,...,999 Other,"4413 Automotive Parts, Accessories, and Tire S...",4413.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"2095 E. Commercial Street, MERIDIAN, ID",43.609116,-116.368114,1.600101e+14
4,4086377107,4/12/20,1087,PPP,"J-U-B ENGINEERS, INC.","250 S. Beechwood Ave., #201",BOISE,ID,83709.0,7/20/21,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,BOISE DISTRICT OFFICE,e $5-10 million,250 to 499,"250 S. Beechwood Ave., #201, BOISE, ID",43.602477,-116.272347,1.600100e+14


Merge with MainFrame

In [18]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,3132717110,4/11/20,1087,PPP,THOMAS MANAGEMENT LLC,700 E FRANKLIN RD,MERIDIAN,ID,83642.0,11/10/21,...,7223 Special Food Services,7223.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"700 E FRANKLIN RD, MERIDIAN, ID",43.605187,-116.382836,1.600101e+14,NaN
1,4037657101,4/12/20,1087,PPP,"CRADLEPOINT, INC.",1111 JEFFERSON ST,BOISE,ID,83702.0,10/21/20,...,3342 Communications Equipment Manufacturing,3342.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"1111 JEFFERSON ST, BOISE, ID",43.619731,-116.205940,1.600100e+14,NaN
2,4034667109,4/12/20,1087,PPP,"ENGINEERED STRUCTURES, INC.","3330 E. Louise Dr, Suite 300",MERIDIAN,ID,83642.0,7/15/21,...,2362 Nonresidential Building Construction,2362.0,BOISE DISTRICT OFFICE,e $5-10 million,250 to 499,"3330 E. Louise Dr, Suite 300, MERIDIAN, ID",43.604024,-116.353172,1.600101e+14,NaN
3,3029957103,4/11/20,1087,PPP,"COMMERCIAL TIRE, INC.",2095 E. Commercial Street,MERIDIAN,ID,83642.0,6/17/21,...,"4413 Automotive Parts, Accessories, and Tire S...",4413.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"2095 E. Commercial Street, MERIDIAN, ID",43.609116,-116.368114,1.600101e+14,NaN
4,4086377107,4/12/20,1087,PPP,"J-U-B ENGINEERS, INC.","250 S. Beechwood Ave., #201",BOISE,ID,83709.0,7/20/21,...,"5413 Architectural, Engineering, and Related S...",5413.0,BOISE DISTRICT OFFICE,e $5-10 million,250 to 499,"250 S. Beechwood Ave., #201, BOISE, ID",43.602477,-116.272347,1.600100e+14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4411,8528738503,3/10/21,1094,PPS,WHITCO INC.,108 Airport Rd,Kamiah,ID,83536.0,10/14/21,...,4842 Specialized Freight Trucking,4842.0,SPOKANE BRANCH OFFICE,"a $150,000-350,000",5 to 9,"108 Airport Rd, Kamiah, ID",46.218843,-116.016474,NaN,160499400002003
4412,8585757008,4/8/20,1094,PPP,"POST FALLS FAMILY DENTAL CENTER, PA",313 N Spokane Street,POST FALLS,ID,83854.0,5/6/21,...,6212 Offices of Dentists,6212.0,SPOKANE BRANCH OFFICE,"a $150,000-350,000",10 to 19,"313 N Spokane Street, POST FALLS, ID",47.710015,-116.948526,NaN,160550004024040
4413,9002167110,4/15/20,1087,PPP,ROBERTS ELECTRIC INC,PO BOX 3869,HAILEY,ID,83333.0,1/8/21,...,2382 Building Equipment Contractors,2382.0,BOISE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"PO BOX 3869, HAILEY, ID",43.519629,-114.315325,NaN,160139601023008
4414,9060577102,4/15/20,1094,PPP,DEI INCORPORATED,13403 W GOVERNMENT WAY,HAYDEN,ID,83835.0,4/8/21,...,2382 Building Equipment Contractors,2382.0,SPOKANE BRANCH OFFICE,"a $150,000-350,000",20 to 49,"13403 W GOVERNMENT WAY, HAYDEN, ID",47.793812,-116.787280,NaN,160550002022016


<b>Fill FIPS Columns - if FIPS file

In [19]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,3132717110,4/11/20,1087,PPP,THOMAS MANAGEMENT LLC,700 E FRANKLIN RD,MERIDIAN,ID,83642.0,11/10/21,...,7223.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"700 E FRANKLIN RD, MERIDIAN, ID",43.605187,-116.382836,1.600101e+14,NaN,160010103682062.0
1,4037657101,4/12/20,1087,PPP,"CRADLEPOINT, INC.",1111 JEFFERSON ST,BOISE,ID,83702.0,10/21/20,...,3342.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"1111 JEFFERSON ST, BOISE, ID",43.619731,-116.205940,1.600100e+14,NaN,160010001021053.0
2,4034667109,4/12/20,1087,PPP,"ENGINEERED STRUCTURES, INC.","3330 E. Louise Dr, Suite 300",MERIDIAN,ID,83642.0,7/15/21,...,2362.0,BOISE DISTRICT OFFICE,e $5-10 million,250 to 499,"3330 E. Louise Dr, Suite 300, MERIDIAN, ID",43.604024,-116.353172,1.600101e+14,NaN,160010103701013.0
3,3029957103,4/11/20,1087,PPP,"COMMERCIAL TIRE, INC.",2095 E. Commercial Street,MERIDIAN,ID,83642.0,6/17/21,...,4413.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"2095 E. Commercial Street, MERIDIAN, ID",43.609116,-116.368114,1.600101e+14,NaN,160010103692012.0
4,4086377107,4/12/20,1087,PPP,"J-U-B ENGINEERS, INC.","250 S. Beechwood Ave., #201",BOISE,ID,83709.0,7/20/21,...,5413.0,BOISE DISTRICT OFFICE,e $5-10 million,250 to 499,"250 S. Beechwood Ave., #201, BOISE, ID",43.602477,-116.272347,1.600100e+14,NaN,160010020003010.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4411,8528738503,3/10/21,1094,PPS,WHITCO INC.,108 Airport Rd,Kamiah,ID,83536.0,10/14/21,...,4842.0,SPOKANE BRANCH OFFICE,"a $150,000-350,000",5 to 9,"108 Airport Rd, Kamiah, ID",46.218843,-116.016474,NaN,160499400002003,160499400002003
4412,8585757008,4/8/20,1094,PPP,"POST FALLS FAMILY DENTAL CENTER, PA",313 N Spokane Street,POST FALLS,ID,83854.0,5/6/21,...,6212.0,SPOKANE BRANCH OFFICE,"a $150,000-350,000",10 to 19,"313 N Spokane Street, POST FALLS, ID",47.710015,-116.948526,NaN,160550004024040,160550004024040
4413,9002167110,4/15/20,1087,PPP,ROBERTS ELECTRIC INC,PO BOX 3869,HAILEY,ID,83333.0,1/8/21,...,2382.0,BOISE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"PO BOX 3869, HAILEY, ID",43.519629,-114.315325,NaN,160139601023008,160139601023008
4414,9060577102,4/15/20,1094,PPP,DEI INCORPORATED,13403 W GOVERNMENT WAY,HAYDEN,ID,83835.0,4/8/21,...,2382.0,SPOKANE BRANCH OFFICE,"a $150,000-350,000",20 to 49,"13403 W GOVERNMENT WAY, HAYDEN, ID",47.793812,-116.787280,NaN,160550002022016,160550002022016


Drop Extra FIPS Columns

In [20]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,3132717110,4/11/20,1087,PPP,THOMAS MANAGEMENT LLC,700 E FRANKLIN RD,MERIDIAN,ID,83642.0,11/10/21,...,999 Other,7223 Special Food Services,7223.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"700 E FRANKLIN RD, MERIDIAN, ID",43.605187,-116.382836,160010103682062.0
1,4037657101,4/12/20,1087,PPP,"CRADLEPOINT, INC.",1111 JEFFERSON ST,BOISE,ID,83702.0,10/21/20,...,334 Computers and Electronic Products,3342 Communications Equipment Manufacturing,3342.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"1111 JEFFERSON ST, BOISE, ID",43.619731,-116.205940,160010001021053.0
2,4034667109,4/12/20,1087,PPP,"ENGINEERED STRUCTURES, INC.","3330 E. Louise Dr, Suite 300",MERIDIAN,ID,83642.0,7/15/21,...,999 Other,2362 Nonresidential Building Construction,2362.0,BOISE DISTRICT OFFICE,e $5-10 million,250 to 499,"3330 E. Louise Dr, Suite 300, MERIDIAN, ID",43.604024,-116.353172,160010103701013.0
3,3029957103,4/11/20,1087,PPP,"COMMERCIAL TIRE, INC.",2095 E. Commercial Street,MERIDIAN,ID,83642.0,6/17/21,...,999 Other,"4413 Automotive Parts, Accessories, and Tire S...",4413.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"2095 E. Commercial Street, MERIDIAN, ID",43.609116,-116.368114,160010103692012.0
4,4086377107,4/12/20,1087,PPP,"J-U-B ENGINEERS, INC.","250 S. Beechwood Ave., #201",BOISE,ID,83709.0,7/20/21,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,BOISE DISTRICT OFFICE,e $5-10 million,250 to 499,"250 S. Beechwood Ave., #201, BOISE, ID",43.602477,-116.272347,160010020003010.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4411,8528738503,3/10/21,1094,PPS,WHITCO INC.,108 Airport Rd,Kamiah,ID,83536.0,10/14/21,...,999 Other,4842 Specialized Freight Trucking,4842.0,SPOKANE BRANCH OFFICE,"a $150,000-350,000",5 to 9,"108 Airport Rd, Kamiah, ID",46.218843,-116.016474,160499400002003
4412,8585757008,4/8/20,1094,PPP,"POST FALLS FAMILY DENTAL CENTER, PA",313 N Spokane Street,POST FALLS,ID,83854.0,5/6/21,...,999 Other,6212 Offices of Dentists,6212.0,SPOKANE BRANCH OFFICE,"a $150,000-350,000",10 to 19,"313 N Spokane Street, POST FALLS, ID",47.710015,-116.948526,160550004024040
4413,9002167110,4/15/20,1087,PPP,ROBERTS ELECTRIC INC,PO BOX 3869,HAILEY,ID,83333.0,1/8/21,...,999 Other,2382 Building Equipment Contractors,2382.0,BOISE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"PO BOX 3869, HAILEY, ID",43.519629,-114.315325,160139601023008
4414,9060577102,4/15/20,1094,PPP,DEI INCORPORATED,13403 W GOVERNMENT WAY,HAYDEN,ID,83835.0,4/8/21,...,999 Other,2382 Building Equipment Contractors,2382.0,SPOKANE BRANCH OFFICE,"a $150,000-350,000",20 to 49,"13403 W GOVERNMENT WAY, HAYDEN, ID",47.793812,-116.787280,160550002022016


In [21]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
369,5223057107,4/13/20,1087,PPP,MCCURRY ONTARIO MOTORS LLC,1805 SW 4th Ave,FRUITLAND,ID,83619.0,9/25/21,...,999 Other,4411 Automobile Dealers,4411.0,BOISE DISTRICT OFFICE,c $1-2 million,50 to 99,"1805 SW 4th Ave, FRUITLAND, ID",44.025265,-116.991226,NaN
1879,6559698606,3/23/21,1087,PPS,DUSTY'S ELECTRIC INC.,20158 Highway 93,Carey,ID,83320.0,NaN,...,999 Other,2382 Building Equipment Contractors,2382.0,BOISE DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"20158 Highway 93, Carey, ID",46.678546,-114.074737,NaN
2296,6866047004,4/7/20,1087,PPP,"DUSTY'S ELECTRIC, INC.",20158 hwy 93,CAREY,ID,83320.0,4/13/21,...,999 Other,2382 Building Equipment Contractors,2382.0,BOISE DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"20158 hwy 93, CAREY, ID",46.678546,-114.074737,NaN
2479,1944747104,4/10/20,1087,PPP,STAR MERCANTILE INC.,PO Box 39,STAR,ID,83669.0,4/17/21,...,999 Other,4451 Grocery Stores,4451.0,BOISE DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"PO Box 39, STAR, ID",NaN,NaN,NaN
2968,3378157108,4/11/20,1087,PPP,BOLTON CONSTRUCTION LLC,208 Spruce Ave Unit B - 4c,KETCHUM,ID,83340.0,12/18/20,...,999 Other,2361 Residential Building Construction,2361.0,BOISE DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"208 Spruce Ave Unit B - 4c, KETCHUM, ID",NaN,NaN,NaN


Convert to File

In [22]:
fips_ID = fips_merge_drop

In [23]:
fips_ID.to_csv("../../data/state_data/geo/geo_fips/ID_fips_scraped.csv") 

Review & Compare

In [24]:
geo_id_fips = pd.read_csv("../../data/state_data/geo/geo_fips/ID_fips_scraped.csv") 
geo_id_fips.head()

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,3132717110,4/11/20,1087,PPP,THOMAS MANAGEMENT LLC,700 E FRANKLIN RD,MERIDIAN,ID,83642.0,...,999 Other,7223 Special Food Services,7223.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"700 E FRANKLIN RD, MERIDIAN, ID",43.605187,-116.382836,1.600101e+14
1,1,4037657101,4/12/20,1087,PPP,"CRADLEPOINT, INC.",1111 JEFFERSON ST,BOISE,ID,83702.0,...,334 Computers and Electronic Products,3342 Communications Equipment Manufacturing,3342.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"1111 JEFFERSON ST, BOISE, ID",43.619731,-116.205940,1.600100e+14
2,2,4034667109,4/12/20,1087,PPP,"ENGINEERED STRUCTURES, INC.","3330 E. Louise Dr, Suite 300",MERIDIAN,ID,83642.0,...,999 Other,2362 Nonresidential Building Construction,2362.0,BOISE DISTRICT OFFICE,e $5-10 million,250 to 499,"3330 E. Louise Dr, Suite 300, MERIDIAN, ID",43.604024,-116.353172,1.600101e+14
3,3,3029957103,4/11/20,1087,PPP,"COMMERCIAL TIRE, INC.",2095 E. Commercial Street,MERIDIAN,ID,83642.0,...,999 Other,"4413 Automotive Parts, Accessories, and Tire S...",4413.0,BOISE DISTRICT OFFICE,e $5-10 million,500 or more,"2095 E. Commercial Street, MERIDIAN, ID",43.609116,-116.368114,1.600101e+14
4,4,4086377107,4/12/20,1087,PPP,"J-U-B ENGINEERS, INC.","250 S. Beechwood Ave., #201",BOISE,ID,83709.0,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,BOISE DISTRICT OFFICE,e $5-10 million,250 to 499,"250 S. Beechwood Ave., #201, BOISE, ID",43.602477,-116.272347,1.600100e+14


In [25]:
##### fill in missing Lat, Long, FIPS
geo_id_fips.loc[geo_id_fips["FIPS_z"].isnull()] #27

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
369,369,5223057107,4/13/20,1087,PPP,MCCURRY ONTARIO MOTORS LLC,1805 SW 4th Ave,FRUITLAND,ID,83619.0,...,999 Other,4411 Automobile Dealers,4411.0,BOISE DISTRICT OFFICE,c $1-2 million,50 to 99,"1805 SW 4th Ave, FRUITLAND, ID",44.025265,-116.991226,NaN
1879,1879,6559698606,3/23/21,1087,PPS,DUSTY'S ELECTRIC INC.,20158 Highway 93,Carey,ID,83320.0,...,999 Other,2382 Building Equipment Contractors,2382.0,BOISE DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"20158 Highway 93, Carey, ID",46.678546,-114.074737,NaN
2296,2296,6866047004,4/7/20,1087,PPP,"DUSTY'S ELECTRIC, INC.",20158 hwy 93,CAREY,ID,83320.0,...,999 Other,2382 Building Equipment Contractors,2382.0,BOISE DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"20158 hwy 93, CAREY, ID",46.678546,-114.074737,NaN
2479,2479,1944747104,4/10/20,1087,PPP,STAR MERCANTILE INC.,PO Box 39,STAR,ID,83669.0,...,999 Other,4451 Grocery Stores,4451.0,BOISE DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"PO Box 39, STAR, ID",NaN,NaN,NaN
2968,2968,3378157108,4/11/20,1087,PPP,BOLTON CONSTRUCTION LLC,208 Spruce Ave Unit B - 4c,KETCHUM,ID,83340.0,...,999 Other,2361 Residential Building Construction,2361.0,BOISE DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"208 Spruce Ave Unit B - 4c, KETCHUM, ID",NaN,NaN,NaN


In [39]:
geo_id_fips["BusinessType"] = geo_id_fips["BusinessType"].fillna('Not Available')
geo_id_fips["BusinessType"].unique()
geo_id_fips.to_csv("../../data/state_data/geo/geo_fips/ID_fips_scraped.csv") 
geo_id_fips =pd.read_csv("../../data/state_data/geo/geo_fips/ID_fips_scraped.csv") 
geo_id_fips["BusinessType"].unique()

array(['Limited Liability Company(LLC)', 'Subchapter S Corporation',
       'Corporation', 'Partnership', 'Limited Liability Partnership',
       '501(c)3 – Non Profit', 'Employee Stock Ownership Plan(ESOP)',
       'Professional Association', 'Sole Proprietorship',
       'Non-Profit Organization', 'Not Available', 'Joint Venture',
       'Cooperative', '501(c)6 – Non Profit Membership',
       'Self-Employed Individuals', 'Trust', 'Independent Contractors',
       'Non-Profit Childcare Center', 'Single Member LLC',
       'Tenant in Common', 'Qualified Joint-Venture (spouses)'],
      dtype=object)